In [ ]:
%pip install geopandas plotly seaborn

In [ ]:
import math
from datetime import datetime
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio

In [ ]:
from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

In [ ]:
PROJECT_NAME = "AreaVerde"

In [ ]:
sel_date_min = datetime(2024, 6, 11,  0,  0,  0)
sel_date_max = datetime(2024, 9, 30, 23, 59, 59)

In [ ]:
# Load data
project = dh.get_or_create_project(PROJECT_NAME)

spira = project.get_dataitem("spira_flow_data_2024").as_df()
spira['DateTime'] = pd.to_datetime(spira['DateTime'])

In [ ]:
spira

In [ ]:
spira_pivot = spira.pivot_table(index='DateTime', columns='spira_code', values='count')

In [ ]:
spira_pivot

In [ ]:
hourly_total_sel = spira_pivot.copy()
hourly_total_sel = hourly_total_sel[sel_date_min:sel_date_max]
hourly_total_sel = hourly_total_sel[hourly_total_sel.columns[hourly_total_sel.max() > 0]]
hourly_total_subsel = hourly_total_sel.iloc[:,:100]

In [ ]:
hourly_total_sel_norm = (hourly_total_sel - hourly_total_sel.mean()) / hourly_total_sel.std()
hourly_total_subsel_norm = (hourly_total_subsel - hourly_total_subsel.mean()) / hourly_total_subsel.std()

In [ ]:
plt.figure(figsize=(30, 30))
heatmap = sns.heatmap(hourly_total_subsel_norm.corr(), vmin=-1, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Spire Correlation', fontdict={'fontsize':20}, pad=16);

In [ ]:
gates_location = project.get_dataitem("gates").as_df()
gates = project.get_dataitem("gate_data").as_df()
# reconvert string to datetime
gates["Data"] = pd.to_datetime(gates["Data"])

In [ ]:
total_vehicles = gates[["Data", "gate", "count"]].groupby(["Data", "gate"], as_index=False).sum()
total_vehicles = total_vehicles.pivot_table(index="Data", columns="gate", values="count", fill_value=0).astype(int)
total_vehicles

In [ ]:
sel_gates = ['Alberto Mario', 'Arcoveggio', 'Corelli_1 Nord', 'Don Sturzo',
             'Dozza', 'Massarenti', 'Saragat', 'Triumvirato', 'Zanardi']

In [ ]:
total_vehicles_sel = total_vehicles.copy()
total_vehicles_sel = total_vehicles_sel[sel_date_min:sel_date_max][sel_gates]

In [ ]:
total_vehicles_sel_norm = (total_vehicles_sel - total_vehicles_sel.mean()) / total_vehicles_sel.std()

In [ ]:
heatmap = sns.heatmap(total_vehicles_sel_norm.corr(), vmin=-1, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Gate Correlation', fontdict={'fontsize':15}, pad=16);

In [ ]:
gates = [['Arcoveggio','Alberto Mario'],['Dozza','Massarenti'],['Alberto Mario','Corelli_1 Nord'],['Arcoveggio','Corelli_1 Nord']]
days = 0
axes = len(gates)
fig, axs = plt.subplots(axes, 1, layout='constrained', figsize=(12, 3*axes))
for i,gs in enumerate(gates):
    for g in gs:
        axs[i].plot(total_vehicles_sel_norm.iloc[:24*days if days else -1][g], label=g)
    axs[i].legend(loc='upper right')
    axs[i].set_title("-".join(gs) + ": " + str(total_vehicles_sel_norm.corr()[gs[0]][gs[1]]))


In [ ]:
total_vehicles_sel_norm_trend = total_vehicles_sel_norm.mean(axis=1)
total_vehicles_sel_norm_detrend = total_vehicles_sel_norm.sub(total_vehicles_sel_norm_trend, axis=0)

In [ ]:
days = 0
total_vehicles_sel_norm_trend.iloc[:24*days if days else -1].plot(figsize=(12,3));

In [ ]:
heatmap = sns.heatmap(total_vehicles_sel_norm_detrend.corr(), vmin=-1, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Gate Correlation', fontdict={'fontsize':15}, pad=16);

In [ ]:
gates = [['Arcoveggio','Alberto Mario'],['Dozza','Massarenti'],['Alberto Mario','Corelli_1 Nord'],['Arcoveggio','Corelli_1 Nord']]
days = 0
axes = len(gates)
fig, axs = plt.subplots(axes, 1, layout='constrained', figsize=(12, 3*axes))
for i,gs in enumerate(gates):
    for g in gs:
        axs[i].plot(total_vehicles_sel_norm_detrend.iloc[:24*days if days else -1][g], label=g)
    axs[i].legend(loc='upper right')
    axs[i].set_title("-".join(gs) + ": " + str(total_vehicles_sel_norm_detrend.corr()[gs[0]][gs[1]]))

In [ ]:
merged_subsel = pd.concat([hourly_total_subsel_norm, total_vehicles_sel_norm], axis=1)
merged_subsel

In [ ]:
ncols_subsel = len(hourly_total_subsel_norm.columns)
corr_subsel = merged_subsel.corr().iloc[:ncols_subsel,ncols_subsel:]
corr_subsel

In [ ]:
plt.figure(figsize=(15, 20))
heatmap = sns.heatmap(corr_subsel, vmin=-1, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Overall Correlation', fontdict={'fontsize':20}, pad=16);

In [ ]:
merged = pd.concat([hourly_total_sel_norm, total_vehicles_sel_norm], axis=1)
merged

In [ ]:
ncols = len(hourly_total_sel_norm.columns)
corr = merged.corr().iloc[:ncols,ncols:]

In [ ]:
columns = corr.idxmax().values
corr = corr.loc[columns]
corr

In [ ]:
vmin = math.floor(corr.min().min() * 20) / 20
heatmap = sns.heatmap(corr, vmin=vmin, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Overall Correlation (max spire)', fontdict={'fontsize':20}, pad=16);

In [ ]:
sel_spira = corr.index
pos_spira = (spira[spira['spira_code'].isin(sel_spira)][['spira_code','longitudine', 'latitudine']].drop_duplicates().rename(columns={'spira_code': 'codice'}))
pos_spira

In [ ]:
sel_gates = corr.columns
pos_gates = gates_location[['gate', 'Longitudine', 'Latitudine']].rename(columns={'gate': 'code', 'Longitudine':'longitudine', 'Latitudine':'latitudine'})
pos_gates = pos_gates[pos_gates['code'].isin(sel_gates)]
pos_gates

In [ ]:
data_points = pd.concat([pos_spira,gates_location])
data_points['size'] = 1
color_list = ['red', 'blue', 'green', 'goldenrod', 'darkred', 'aqua', 'navy', 'magenta', 'dimgrey' ]

color_map_s = { s: color_list[i] for i,s in enumerate(sel_spira) }
color_map_v = { v: color_list[i] for i,v in enumerate(sel_gates) }
color_map = dict(color_map_s, **color_map_v)

fig = px.scatter_mapbox(
        data_points,
        lat='latitudine',
        lon='longitudine',
        color='codice',
        color_discrete_map=color_map,
        mapbox_style='open-street-map',
        size='size',
        size_max=12,
        zoom=11.75,
        height=800,
    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()